In [133]:
import glob
import pandas as pd
import time
import numpy as np

# Creating Sparse Matrix

In [106]:
# Reading in all the results
paths = glob.glob('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/Query results/*.csv')

dfs = [pd.read_csv(filename, index_col=0) for filename in paths]

df = pd.concat(dfs,ignore_index=True)

In [107]:
locations = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/locations_updated.csv',index_col=0)
locations['Address'] = locations.index
locations = locations[['long','lat']]

In [108]:
# Reading in the list of new locations
extras = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/poco-geocodedataExtraLocations.csv',index_col=5)
extras = extras[['longitude','latitude']]
extras = extras.rename(columns={'longitude':'long','latitude':'lat'})

In [109]:
# Creating master list of locations
locationsComplete = pd.concat([locations,extras])

In [110]:
locationsSet = set(locationsComplete.index)

In [112]:
# Creating a locations dataframe, each locations has a unique, in order index
locationsDF = pd.DataFrame({'locations':list(locationsSet)})
locationsDF['location'] = locationsDF.index

In [113]:
end = df[['time','end']]
end = pd.merge(end, locationsDF,how='left', left_on='end', right_on='locations')[['end','location']]
end = end.rename(columns={'location':'endLocation'})

In [114]:
start = df[['time','start']]
start = pd.merge(start, locationsDF,how='left', left_on='start', right_on='locations')[['time','start','location']]
start = start.rename(columns={'location':'startLocation'})

In [120]:
indexedQueries = pd.DataFrame({'time':start['time'],
              'start':start['start'],
              'startLocation':start['startLocation'],
              'end':end['end'],
              'endLocation':end['endLocation']})

In [168]:
timeMatrix = np.empty((len(locationsDF),len(locationsDF)))
timeMatrix[:] = np.nan

In [169]:
i = 0
total = len(indexedQueries)
t0 = time.time()
for rowNum, row in indexedQueries.iterrows():
    timeMatrix[row['startLocation'], row['endLocation']] = row['time']
    timeMatrix[row['endLocation'], row['startLocation']] = row['time']
    
    i += 1
    if i % 10000 == 0:
        print(round(100*i/total), '% Complete')   

3 % Complete
5 % Complete
8 % Complete
10 % Complete
13 % Complete
15 % Complete
18 % Complete
20 % Complete
23 % Complete
26 % Complete
28 % Complete
31 % Complete
33 % Complete
36 % Complete
38 % Complete
41 % Complete
44 % Complete
46 % Complete
49 % Complete
51 % Complete
54 % Complete
56 % Complete
59 % Complete
61 % Complete
64 % Complete
67 % Complete
69 % Complete
72 % Complete
74 % Complete
77 % Complete
79 % Complete
82 % Complete
84 % Complete
87 % Complete
90 % Complete
92 % Complete
95 % Complete
97 % Complete
100 % Complete


In [170]:
i = 0
for i in range(0,len(timeMatrix)):
    timeMatrix[i,i] = 0
    i += 1
    if i % 10000 == 0:
        print(round(100*i/total), '% Complete')

3 % Complete


In [175]:
timeMatrixDF = pd.DataFrame(timeMatrix, index = locationsDF['locations'],columns = locationsDF['locations'])

In [182]:
timeMatrixDF.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/sparseTimeMatrixUpdated.csv')